In [62]:
import requests
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re
import numpy as np

---
## 1. Thu thập dữ liệu từ web Zillow bằng cách parse HTML 
File này gồm có các cột:
- Address: Địa chỉ nhà.
- Price: Gía nhà (là giá của nhà đã bán).
- Bedrooms: số phòng ngủ.
- Bathrooms: số phòng tắm.
- Living Area: Diện tích căn nhà (sqft).
- Heating : Hệ thống sưởi.
- Basement : Tầng hầm(Yes/No).
- Yearbuild: Năm xây dựng.
- Fireplace: Lò sưởi(Yes/No).
- Garage: Nhà xe.


In [38]:
Total_pages=10
req_headers = {
    'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'accept-encoding':'gzip, deflate, sdch, br',
    'accept-language':'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
    'cache-control':'max-age=0',
    'upgrade-insecure-requests':'1',
    'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
courses =pd.DataFrame()
Address_ =[]
Price_=[]
Bedrooms_=[]
Bathrooms_ =[]
LivingArea_ = []
Heating_ =[]
Basement_ =[]
Yearbuild_ =[]
Fireplace_ =[]
Garage_ =[]
with requests.Session() as s:
    page =1
    url = 'https://www.zillow.com/manchester-ct/sold/?searchQueryState=%7B"pagination"%3A%7B%7D%2C"usersSearchTerm"%3A"Manchester%2C%20CT"%2C"mapBounds"%3A%7B"west"%3A-72.62056917797851%2C"east"%3A-72.42659182202148%2C"south"%3A41.72559685174378%2C"north"%3A41.82852106786825%7D%2C"regionSelection"%3A%5B%7B"regionId"%3A398963%2C"regionType"%3A6%7D%5D%2C"isMapVisible"%3Atrue%2C"filterState"%3A%7B"fsba"%3A%7B"value"%3Afalse%7D%2C"fsbo"%3A%7B"value"%3Afalse%7D%2C"nc"%3A%7B"value"%3Afalse%7D%2C"fore"%3A%7B"value"%3Afalse%7D%2C"cmsn"%3A%7B"value"%3Afalse%7D%2C"auc"%3A%7B"value"%3Afalse%7D%2C"pmf"%3A%7B"value"%3Afalse%7D%2C"pf"%3A%7B"value"%3Afalse%7D%2C"rs"%3A%7B"value"%3Atrue%7D%2C"ah"%3A%7B"value"%3Atrue%7D%7D%2C"isListVisible"%3Atrue%2C"mapZoom"%3A13%7D'
    while(page < Total_pages):
        time.sleep(1)
        r = s.get(url, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        detailLink = soup.find_all('a',{'class':'list-card-link list-card-link-top-margin list-card-img'})
        prices = soup.find_all('div',{'class':'list-card-price'})
        for price in prices:
            Price_.append(price.text)
        address = soup.find_all('address',{'class':'list-card-addr'})
        for addr in address:
            Address_.append(addr.text)
        page+=1
        url ='https://www.zillow.com/manchester-nh/sold/' +str(page)+'_p/'
        for link in detailLink:
            detail = s.get(link['href'],headers=req_headers)
            soup1 = BeautifulSoup(detail.content, 'html.parser')
            bbl = soup1.find_all('span',{'class':'ds-bed-bath-living-area'},limit =3) # bed-bad-livingarea
            bed_bad_livingarea =''
            for i in bbl:
                bed_bad_livingarea+= i.text +'\t'
            Bedrooms_.append(bed_bad_livingarea.split('\t')[0])
            Bathrooms_.append(bed_bad_livingarea.split('\t')[1])
            LivingArea_.append(bed_bad_livingarea.split('\t')[2])
            heating = soup1.find("span",string=re.compile("Heating:"))
            if heating is not None:
                heating = heating.find_next_sibling('span').text
                Heating_.append(heating)
            else:
                Heating_.append('None')
            basement = soup1.find("span",string=re.compile("Basement"))
            if basement is not None:
                if ':' in basement.find_next('span').text:
                    basement = basement.find_next('span').text.split(':')[1]
                    Basement_.append(basement)
                else:
                    Basement_.append('None')
            else:
                    Basement_.append('None')
            yearbuild = soup1.find("span",string=re.compile("Condition"))
            if yearbuild is not None:
                if ':' in yearbuild.find_next('span').text:
                    yearbuild = yearbuild.find_next('span').text.split(': ')[1]
                    Yearbuild_.append(yearbuild)
                else:
                    Yearbuild_.append('None')
            else:
                    Yearbuild_.append('None')

            fireplace = soup1.find("h5",string=re.compile("Property details"))
            if fireplace is not None:
                if 'Fireplace:' in fireplace.find_previous('span').text:
                    fireplace = fireplace.find_previous('span').text.split(': ')[1]
                    Fireplace_.append(fireplace)
                else:
                    Fireplace_.append('None')
            else:
                    Fireplace_.append('None')

            garage = soup1.find("span",string=re.compile("Property"))
            if garage is not None:
                if 'Garage spaces:' in garage.find_previous('span').text:
                    garage = garage.find_previous('span').text.split(': ')[1]
                    Garage_.append(garage)
                else:
                    Garage_.append('None')
            else:
                    Garage_.append('None')

courses['Address'] = Address_
courses['Bedrooms'] = Bedrooms_
courses['Bathrooms'] = Bathrooms_
courses['LivingArea'] = LivingArea_
courses['Heating'] = Heating_
courses['Basement'] = Basement_
courses['YearBuild'] = Yearbuild_
courses['Fireplace'] = Fireplace_
courses['Garage'] = Garage_
courses['Price'] = Price_
courses.to_csv('courses.csv')

---
## 2. Khám phá dữ liệu

In [108]:
# Đọc dữ liệu
House_df = pd.read_csv('courses.csv',index_col=0)
House_df.head()

,Address,Bedrooms,Bathrooms,LivingArea,Heating,Basement,YearBuild,Fireplace,Garage,Price
0,"216 Walek Farms Rd, Manchester, CT 06040",3 bd,3 ba,"2,556 sqft","Baseboard, Gas",Finished,1995,Yes,2,"$300,000"
1,"39 S Hawthorne St, Manchester, CT 06040",3 bd,1 ba,"1,080 sqft","Forced air, Gas",Partially finished,1945,None,0,"$170,000"
2,"16 Saddle Hill Rd, Manchester, CT 06040",4 bd,4 ba,"3,832 sqft",Forced air,Finished,2007,Yes,2,"$447,900"
3,"99 Pond Ln, Manchester, CT 06042",4 bd,3 ba,"1,852 sqft","Forced air, Oil",NaN,1964,Yes,3,"$299,000"
4,"430 E Center St #430, Manchester, CT 06040",1 bd,1.5 ba,"1,074 sqft","Forced air, Gas",None,1976,None,0,"$1,200"


### Shape của dữ liệu

In [109]:
House_df.shape

(360, 10)

In [110]:
# Cột output hiện có kiểu dữ liệu gì?
House_df['Price'].dtype

dtype('O')

In [111]:
# Cột output có giá trị thiếu không?
House_df['Price'].isna().sum()

0

In [112]:
# xem thêm mô tả về dữ liệu
House_df.describe()

,Address,Bedrooms,Bathrooms,LivingArea,Heating,Basement,YearBuild,Fireplace,Garage,Price
count,360,360,360,360,360,358,360,360,360,360
unique,349,13,10,298,67,20,104,2,9,208
top,"503 Granite St, Manchester, NH 03102",3 bd,2 ba,"1,200 sqft","Forced air, Gas",Yes,1950,None,0,"$380,000"
freq,2,146,121,5,55,191,20,281,132,8


- Ta thấy cột Address có giá trị xuất hiện 2 lần, có nghĩa là có giá trị bị lặp
- Điều này là do zillow thường render 1 cách random(theo cách nào đó) các ngôi nhà ĐÃ BÁN -> bị lặp

In [113]:
# kiểm tra dữ liệu xem bị lặp bao nhiêu lần
House_df.Address.duplicated().sum()

11

---
## 3. Tiền xử lý dữ liệu


### Xử lí thô


In [114]:
# như đã khám phá, ta biết dữ liệu bị lặp, ta sẽ loại bỏ những giá trị lặp
House_df.drop_duplicates(subset=['Address'])

,Address,Bedrooms,Bathrooms,LivingArea,Heating,Basement,YearBuild,Fireplace,Garage,Price
0,"216 Walek Farms Rd, Manchester, CT 06040",3 bd,3 ba,"2,556 sqft","Baseboard, Gas",Finished,1995,Yes,2,"$300,000"
1,"39 S Hawthorne St, Manchester, CT 06040",3 bd,1 ba,"1,080 sqft","Forced air, Gas",Partially finished,1945,None,0,"$170,000"
2,"16 Saddle Hill Rd, Manchester, CT 06040",4 bd,4 ba,"3,832 sqft",Forced air,Finished,2007,Yes,2,"$447,900"
3,"99 Pond Ln, Manchester, CT 06042",4 bd,3 ba,"1,852 sqft","Forced air, Oil",NaN,1964,Yes,3,"$299,000"
4,"430 E Center St #430, Manchester, CT 06040",1 bd,1.5 ba,"1,074 sqft","Forced air, Gas",None,1976,None,0,"$1,200"
...,...,...,...,...,...,...,...,...,...,...
355,"3 Country Club Dr APT 303, Manchester, NH 03102",2 bd,2 ba,"1,468 sqft","Forced air, Electric, Gas",None,2005,Yes,1,"$360,000"
356,"171 Laval St, Manchester, NH 03102",3 bd,2 ba,"1,638 sqft",Gas,Yes,1890,None,1,"$274,933"
357,"467 Hevey St, Manchester, NH 03102",8 bd,3 ba,"3,200 sqft","Forced air, Electric, Gas",Unfinished,1896,None,6,"$345,777"
358,"40 Green Meadow Ln, Manchester, NH 03109",4 bd,3 ba,"2,116 sqft","Forced air, Gas",Yes,1999,None,2,"$392,000"


In [115]:
# tiền xử lý cột bedrooms
# Loại bỏ kí tự "bd", giữ lại số
# với những giá trị thiếu, ví dụ "-- bd" ta thay bằng np.nan
for idx,value in House_df['Bedrooms'].items():
    num = value.split(' ')[0]
    if num == '--':
        House_df['Bedrooms'][idx]= np.nan
    else:
        House_df['Bedrooms'][idx]= num
House_df['Bedrooms'].unique()

array(['3', '4', '1', nan, '2', '6', '7', '5', '10', '12', '18', '8', '9'],
      dtype=object)

In [116]:
# tiền xử lý cột Bathrooms
# Loại bỏ kí tự "ba", giữ lại số
# với những giá trị thiếu, ví dụ "-- ba" ta thay bằng np.nan
for idx,value in House_df['Bathrooms'].items():
    num = value.split(' ')[0]
    if num == '--':
        House_df['Bathrooms'][idx]= np.nan
    else:
        House_df['Bathrooms'][idx]= num
House_df['Bathrooms'].unique()

array(['3', '1', '4', '1.5', '5', '2', '2.5', nan, '3.5', '1.75'],
      dtype=object)

In [117]:
# tiền xử lý cột Livingarea
# Loại bỏ kí tự "sqft", giữ lại số
# với những giá trị thiếu, ví dụ "-- sqft" ta thay bằng np.nan
for idx,value in House_df['LivingArea'].items():
    num = value.split(' ')[0]
    if num == '--':
        House_df['LivingArea'][idx]= np.nan
    else:
        House_df['LivingArea'][idx]= num
House_df['LivingArea']

0      2,556
1      1,080
2      3,832
3      1,852
4      1,074
       ...  
355    1,468
356    1,638
357    3,200
358    2,116
359    1,845
Name: LivingArea, Length: 360, dtype: object

In [118]:
# tiền xử lý cột Basement
# Loại bỏ kí tự "None" hay "" và thay = np.nan
for idx,value in House_df['Basement'].items():
    if value == 'None' or value =="":
        House_df['Basement'][idx]= np.nan
House_df['Basement']

0                 Finished
1       Partially finished
2                 Finished
3                      NaN
4                      NaN
              ...         
355                    NaN
356                    Yes
357             Unfinished
358                    Yes
359                    Yes
Name: Basement, Length: 360, dtype: object

In [127]:
# tiền xử lý cột YearBuild
# thay thế những giá trị không phải là số = np.nan
House_df['YearBuild'] = pd.to_numeric(House_df['YearBuild'],errors='coerce')
House_df['YearBuild'].unique()

array([1995., 1945., 2007., 1964., 1976., 1900., 1950., 1959., 1951.,
       2002., 1954., 1925., 1965., 1952., 1944., 1942., 1940., 1930.,
       1913.,   nan, 1922., 1896., 1927., 1962., 1910., 1918., 1941.,
       1981., 1998., 1977., 1938., 1990., 1890., 1943., 2005., 1889.,
       1960., 1974., 1901., 2001., 1979., 1978., 2018., 1948., 2020.,
       1885., 1967., 1983., 1920., 1961., 1982., 2006., 1957., 1985.,
       1947., 1975., 1915., 2014., 1973., 1880., 1800., 1986., 1955.,
       1972., 1903., 1921., 1980., 1997., 1935., 1969., 1984., 2004.,
       1850., 1946., 1893., 1898., 1931., 1988., 1999., 1911., 2008.,
       1958., 1881., 2000., 1993., 1908., 1917., 1970., 1934., 1971.,
       2009., 1956., 2016., 1987., 1992., 1904., 1887., 1875., 1914.,
       1912., 1869., 2019.])

In [128]:
# tiền xử lý cột YearBuild
# thay thế những giá trị không phải là số = np.nan
House_df['Garage'] = pd.to_numeric(House_df['Garage'],errors='coerce')
House_df['Garage'].unique()

array([ 2.,  0.,  3.,  4., nan,  1.,  6.,  5.,  8.])

In [131]:
# tiền xử lý cột Price
# loại bỏ kí tự $
for idx,value in House_df['Price'].items():
    num = value.split('$')[1]
    House_df['Price'][idx]= num
House_df['Price']

<ipython-input-131-1f91ff093950>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Price'][idx]= num


0      300,000
1      170,000
2      447,900
3      299,000
4        1,200
        ...   
355    360,000
356    274,933
357    345,777
358    392,000
359    342,533
Name: Price, Length: 360, dtype: object

In [133]:
# tiền xử lý cột Fireplace
# Loại bỏ kí tự "None" hay "" và thay = np.nan
for idx,value in House_df['Fireplace'].items():
    if value == 'None' or value =="":
        House_df['Fireplace'][idx]= np.nan
House_df['Fireplace']

<ipython-input-133-68a20b37200f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Fireplace'][idx]= np.nan


0      Yes
1      NaN
2      Yes
3      Yes
4      NaN
      ... 
355    Yes
356    NaN
357    NaN
358    NaN
359    Yes
Name: Fireplace, Length: 360, dtype: object

---
#### Hoàn tất xử lí dữ liệu thô
